In [15]:
from oemof.solph import create_time_index
import os

import pandas as pd

from InRetEnsys import InRetEnsysBus, InRetEnsysFlow, InRetEnsysModel, InRetEnsysSink, InRetEnsysSource, \
    InRetEnsysEnergysystem, InRetEnsysStorage, InRetEnsysInvestment, InRetEnsysTransformer, Frequencies

# Read data file
filename = os.path.join(
    os.path.dirname(os.getcwd()), "notebooks", "input_data.csv"
)
data = pd.read_csv(filename)

solver = "cbc"

# Create an energy system and optimize the dispatch at least costs.
# ####################### initialize and provide data #####################
energysystem = InRetEnsysEnergysystem(
    time_steps=8760,
    start_date="01/01/2025",
    frequenz=Frequencies.hourly
)

# ######################### create energysystem components ################
# resource buses
bcoal = InRetEnsysBus(label="coal", balanced=False)
bgas = InRetEnsysBus(label="gas", balanced=False)
boil = InRetEnsysBus(label="oil", balanced=False)
blig = InRetEnsysBus(label="lignite", balanced=False)

# electricity and heat
bel = InRetEnsysBus(label="bel")
bth = InRetEnsysBus(label="bth")

energysystem.add(bcoal)
energysystem.add(bgas)
energysystem.add(boil)
energysystem.add(blig)
energysystem.add(bel)
energysystem.add(bth)

# an excess and a shortage variable can help to avoid infeasible problems
energysystem.add(InRetEnsysSink(label="excess_el", inputs={"bel": InRetEnsysFlow()}))
# shortage_el = InRetEnsysSource(label='shortage_el',
#                      outputs={bel: InRetEnsysFlow(variable_costs=200)})

# sources
energysystem.add(
    InRetEnsysSource(
        label="wind",
        outputs={"bel": InRetEnsysFlow(
            # fix=data["wind"],
            nominal_value=66.3
        )},
    )
)

energysystem.add(
    InRetEnsysSource(
        label="pv",
        outputs={"bel": InRetEnsysFlow(
            # fix=data["pv"],
            nominal_value=65.3
        )},
    )
)

# demands (electricity/heat)
energysystem.add(
    InRetEnsysSink(
        label="demand_el",
        inputs={"bel": InRetEnsysFlow(
            nominal_value=85,
            # fix=data["demand_el"]
        )},
    )
)

energysystem.add(
    InRetEnsysSink(
        label="demand_th",
        inputs={"bth": InRetEnsysFlow(
            nominal_value=40,
            # fix=data["demand_th"]
        )},
    )
)

# power plants
energysystem.add(
    InRetEnsysTransformer(
        label="pp_coal",
        inputs={"bcoal": InRetEnsysFlow()},
        outputs={"bel": InRetEnsysFlow(nominal_value=20.2, variable_costs=25)},
        conversion_factors={"bel": 0.39},
    )
)

energysystem.add(
    InRetEnsysTransformer(
        label="pp_lig",
        inputs={"blig": InRetEnsysFlow()},
        outputs={"bel": InRetEnsysFlow(nominal_value=11.8, variable_costs=19)},
        conversion_factors={"bel": 0.41},
    )
)

energysystem.add(
    InRetEnsysTransformer(
        label="pp_gas",
        inputs={"bgas": InRetEnsysFlow()},
        outputs={"bel": InRetEnsysFlow(nominal_value=41, variable_costs=40)},
        conversion_factors={"bel": 0.50},
    )
)

energysystem.add(
    InRetEnsysTransformer(
        label="pp_oil",
        inputs={"boil": InRetEnsysFlow()},
        outputs={"bel": InRetEnsysFlow(nominal_value=5, variable_costs=50)},
        conversion_factors={"bel": 0.28},
    )
)

# combined heat and power plant (chp)
energysystem.add(
    InRetEnsysTransformer(
        label="pp_chp",
        inputs={"bgas": InRetEnsysFlow()},
        outputs={
            "bel": InRetEnsysFlow(nominal_value=30, variable_costs=42),
            "bth": InRetEnsysFlow(nominal_value=40),
        },
        conversion_factors={"bel": 0.3, "bth": 0.4},
    )
)

# heat pump with a coefficient of performance (COP) of 3
b_heat_source = InRetEnsysBus(label="b_heat_source")
energysystem.add(b_heat_source)

energysystem.add(
    InRetEnsysSource(label="heat_source", outputs={"b_heat_source": InRetEnsysFlow()})
)

cop = 3
energysystem.add(
    InRetEnsysTransformer(
        label="heat_pump",
        inputs={"bel": InRetEnsysFlow(), "b_heat_source": InRetEnsysFlow()},
        outputs={"bth": InRetEnsysFlow(nominal_value=10)},
        conversion_factors={"bel": 1 / 3, "b_heat_source": (cop - 1) / cop},
    )
)

print(energysystem.model_dump_json())



{"busses":[{"label":"coal","balanced":false},{"label":"gas","balanced":false},{"label":"oil","balanced":false},{"label":"lignite","balanced":false},{"label":"bel","balanced":true},{"label":"bth","balanced":true},{"label":"b_heat_source","balanced":true}],"sinks":[{"label":"excess_el","inputs":{"bel":{}}},{"label":"demand_el","inputs":{"bel":{"nominal_value":85.0}}},{"label":"demand_th","inputs":{"bth":{"nominal_value":40.0}}}],"sources":[{"label":"wind","outputs":{"bel":{"nominal_value":66.3}}},{"label":"pv","outputs":{"bel":{"nominal_value":65.3}}},{"label":"heat_source","outputs":{"b_heat_source":{}}}],"transformers":[{"label":"pp_coal","inputs":{"bcoal":{}},"outputs":{"bel":{"nominal_value":20.2,"variable_costs":25.0}},"conversion_factors":{"bel":0.39}},{"label":"pp_lig","inputs":{"blig":{}},"outputs":{"bel":{"nominal_value":11.8,"variable_costs":19.0}},"conversion_factors":{"bel":0.41}},{"label":"pp_gas","inputs":{"bgas":{}},"outputs":{"bel":{"nominal_value":41.0,"variable_costs":4